<a href="https://colab.research.google.com/github/jdberkson/OPTI-556-Computational-Imaging/blob/main/FinalProject_JoelBerkson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gamma Distortion Correction

In most display systems, whether it be LED screens, projectors, or even cameras, the brightness response is nonlinear compared to the true brightness desired/observed. This is typically represented as a gamma factor $I^\gamma$, where linear response is $\gamma$ = 1. Usually it is beneficial in display systems to have a gamma factor greater than 1, which will look more natural to our eye, but in some systems this is undesirable. If you desire to recover the inital image.

This is the case in any metrology system that utilizes a phase shifting algorithm (PSA). This includes Phase Shifting Interferometers (PSI), Phase Measuring Deflectometry (PMD), Fringe Projection Profilometry (FPP), and others involving phase shifted patterns.

For these systems the ideal pattern takes the form of:

$$
I_n(x) = A+Bsin(\omega_ox+n*2\pi/N+\phi(x)) \tag{Eq. 1}
$$

Where N is the number of phase shifts, and n denotes the nth pattern, $\omega_o$ is the pattern frequency. The goal of a PSA is to recover $\phi(x)$, which effectively encodes the surface under test with a phase, which is then converted to depth/slope information, depending on the metrology method being used. One of the most common ways to recover this phase is through what's known as a the PSA LSQ (Least Squares):

$$
\phi(x,y) = arctan(\frac{\sum_{n=0}^{N} I_n(x,y)sin(2\pi n/N)}{ \sum_{n=0}^{N} I_n(x,y)cos(2\pi n/N) }) \tag{Eq. 2}
$$

This equation effectively least squares fits a point on the fringe pattern, and can perfectly recover the phase, given a perfect input pattern.
This issue is, the pattern never perfect in real life. There's noise, gamma distortion, saturation, etc. In this project, I will only address gamma distortion. by adding the exponential gamma factor to Eq. 1:

$$
I_n(x)^\gamma = (A+Bsin(\omega_ox+n*2\pi/N+\phi(x)))^\gamma \tag{Eq. 3}
$$

Writing this as a fourier series in exponential form:

$$
I_n(x)^\gamma = A+\sum_{k=1}^{\infty}B_ksin(k[\omega_ox+n*2\pi/N+\phi(x))]) \tag{Eq. 4}
$$

This produces the kth harmonic terms with ampltitude $B_k$. When $\gamma \neq 1$, and the phase shited series from Eq. 1 is used with the PSA LSQ in Eq. 2, the resulting phase will have unwanted ripples at the higher freqeuncy harmonics of the original pattern, which will then affect the final measurement.

There are many methods that already exist to remove gamma distortion, but almost all of them involve designing a frequency filter to remove the higher order harmonics from the phase map, but this method doesn't always work perfect. My goal is that the gamma value can be estimated by training a CNN on just one image of the fringes, and then the inverse can be applied to recover the fringe pattern $I_{actual} = I_{measured}^{1/\gamma}$